# Problem Statement

We are trying to solve

# Import Libraries

In [1]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import SimpleRNN
import pandas as pd
import os

# Load Data

In [2]:
def readfiles(path):
    X = []
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        df = pd.read_csv(file_path)
        X.append(df.values.tolist())
    return X

In [3]:
# Load Data
train = readfiles("./jsb_chorales/train")
test = readfiles("./jsb_chorales/test")
valid = readfiles("./jsb_chorales/valid")


# Build a RNN

## SimpleRNN

In [ ]:
model_rnn = Sequential()
model_rnn.add(
    SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    SimpleRNN(20, return_sequences=True),
    SimpleRNN(1)
)